# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 7 2022 10:17AM,241285,15,8240391,"Laser Pharmaceuticals, LLC",Released
1,Jul 7 2022 10:17AM,241285,15,8240392,"Laser Pharmaceuticals, LLC",Released
2,Jul 7 2022 10:17AM,241285,15,8240393,"Laser Pharmaceuticals, LLC",Released
3,Jul 7 2022 10:17AM,241285,15,8240394,"Laser Pharmaceuticals, LLC",Released
4,Jul 7 2022 10:17AM,241285,15,8240395,"Laser Pharmaceuticals, LLC",Released
5,Jul 7 2022 10:17AM,241285,15,8240396,"Laser Pharmaceuticals, LLC",Released
6,Jul 7 2022 10:17AM,241285,15,8240397,"Laser Pharmaceuticals, LLC",Released
7,Jul 7 2022 10:17AM,241285,15,8240398,"Laser Pharmaceuticals, LLC",Released
8,Jul 7 2022 10:17AM,241285,15,8240399,"Laser Pharmaceuticals, LLC",Released
9,Jul 7 2022 10:17AM,241285,15,8240400,"Laser Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
51,241281,Released,1
52,241282,Released,18
53,241283,Released,12
54,241284,Released,2
55,241285,Released,12


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
241281,NaN,1.0
241282,NaN,18.0
241283,NaN,12.0
241284,NaN,2.0
241285,NaN,12.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
241186,0.0,9.0
241187,0.0,57.0
241188,0.0,69.0
241189,20.0,26.0
241191,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
241186,0,9
241187,0,57
241188,0,69
241189,20,26
241191,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,241186,0,9
1,241187,0,57
2,241188,0,69
3,241189,20,26
4,241191,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,241186,,9
1,241187,,57
2,241188,,69
3,241189,20,26
4,241191,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 7 2022 10:17AM,241285,15,"Laser Pharmaceuticals, LLC"
12,Jul 7 2022 10:11AM,241282,15,"Alliance Pharma, Inc."
30,Jul 7 2022 10:11AM,241284,20,"Exact-Rx, Inc."
32,Jul 7 2022 10:09AM,241283,10,MedStone Pharma LLC
44,Jul 7 2022 10:08AM,241255,15,"Brookfield Pharmaceuticals, LLC"
67,Jul 7 2022 10:07AM,241254,15,"Person & Covey, Inc."
105,Jul 7 2022 10:06AM,241281,10,Emerginnova
106,Jul 7 2022 9:51AM,241280,20,Alumier Labs Inc.
129,Jul 7 2022 9:34AM,241279,10,ISDIN Corporation
150,Jul 7 2022 9:27AM,241278,10,"Nextsource Biotechnology, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 7 2022 10:17AM,241285,15,"Laser Pharmaceuticals, LLC",,12
1,Jul 7 2022 10:11AM,241282,15,"Alliance Pharma, Inc.",,18
2,Jul 7 2022 10:11AM,241284,20,"Exact-Rx, Inc.",,2
3,Jul 7 2022 10:09AM,241283,10,MedStone Pharma LLC,,12
4,Jul 7 2022 10:08AM,241255,15,"Brookfield Pharmaceuticals, LLC",,23
5,Jul 7 2022 10:07AM,241254,15,"Person & Covey, Inc.",,38
6,Jul 7 2022 10:06AM,241281,10,Emerginnova,,1
7,Jul 7 2022 9:51AM,241280,20,Alumier Labs Inc.,,23
8,Jul 7 2022 9:34AM,241279,10,ISDIN Corporation,,21
9,Jul 7 2022 9:27AM,241278,10,"Nextsource Biotechnology, LLC",,7


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 10:17AM,241285,15,"Laser Pharmaceuticals, LLC",12,
1,Jul 7 2022 10:11AM,241282,15,"Alliance Pharma, Inc.",18,
2,Jul 7 2022 10:11AM,241284,20,"Exact-Rx, Inc.",2,
3,Jul 7 2022 10:09AM,241283,10,MedStone Pharma LLC,12,
4,Jul 7 2022 10:08AM,241255,15,"Brookfield Pharmaceuticals, LLC",23,
5,Jul 7 2022 10:07AM,241254,15,"Person & Covey, Inc.",38,
6,Jul 7 2022 10:06AM,241281,10,Emerginnova,1,
7,Jul 7 2022 9:51AM,241280,20,Alumier Labs Inc.,23,
8,Jul 7 2022 9:34AM,241279,10,ISDIN Corporation,21,
9,Jul 7 2022 9:27AM,241278,10,"Nextsource Biotechnology, LLC",7,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 10:17AM,241285,15,"Laser Pharmaceuticals, LLC",12,
1,Jul 7 2022 10:11AM,241282,15,"Alliance Pharma, Inc.",18,
2,Jul 7 2022 10:11AM,241284,20,"Exact-Rx, Inc.",2,
3,Jul 7 2022 10:09AM,241283,10,MedStone Pharma LLC,12,
4,Jul 7 2022 10:08AM,241255,15,"Brookfield Pharmaceuticals, LLC",23,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 10:17AM,241285,15,"Laser Pharmaceuticals, LLC",12.0,NaN
1,Jul 7 2022 10:11AM,241282,15,"Alliance Pharma, Inc.",18.0,NaN
2,Jul 7 2022 10:11AM,241284,20,"Exact-Rx, Inc.",2.0,NaN
3,Jul 7 2022 10:09AM,241283,10,MedStone Pharma LLC,12.0,NaN
4,Jul 7 2022 10:08AM,241255,15,"Brookfield Pharmaceuticals, LLC",23.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 10:17AM,241285,15,"Laser Pharmaceuticals, LLC",12.0,0.0
1,Jul 7 2022 10:11AM,241282,15,"Alliance Pharma, Inc.",18.0,0.0
2,Jul 7 2022 10:11AM,241284,20,"Exact-Rx, Inc.",2.0,0.0
3,Jul 7 2022 10:09AM,241283,10,MedStone Pharma LLC,12.0,0.0
4,Jul 7 2022 10:08AM,241255,15,"Brookfield Pharmaceuticals, LLC",23.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,6996042,498.0,22.0
12,723757,18.0,0.0
15,965076,91.0,0.0
16,723709,1.0,2.0
19,482482,63.0,0.0
20,964982,50.0,4.0
21,1688737,7.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,6996042,498.0,22.0
1,12,723757,18.0,0.0
2,15,965076,91.0,0.0
3,16,723709,1.0,2.0
4,19,482482,63.0,0.0
5,20,964982,50.0,4.0
6,21,1688737,7.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,498.0,22.0
1,12,18.0,0.0
2,15,91.0,0.0
3,16,1.0,2.0
4,19,63.0,0.0
5,20,50.0,4.0
6,21,7.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,498.0
1,12,Released,18.0
2,15,Released,91.0
3,16,Released,1.0
4,19,Released,63.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Executing,22.0,0.0,0.0,2.0,0.0,4.0,0.0
Released,498.0,18.0,91.0,1.0,63.0,50.0,7.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Executing,22.0,0.0,0.0,2.0,0.0,4.0,0.0
1,Released,498.0,18.0,91.0,1.0,63.0,50.0,7.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Executing,22.0,0.0,0.0,2.0,0.0,4.0,0.0
1,Released,498.0,18.0,91.0,1.0,63.0,50.0,7.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()